## Image Embeddings

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np

from os import listdir, makedirs, path

from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from PIL import Image as PImage, ImageOps as PImageOps

from parameters.arquigrafia import IMAGES_PATH
from parameters.embeddings import EMBEDDINGS_PATH

from models.embedding_models import Clip, EfficientNet, ResNet, Vit

makedirs(EMBEDDINGS_PATH, exist_ok=True)

### Run embeddings

In [ ]:
input_files = sorted([f for f in listdir(IMAGES_PATH) if f.endswith("jpg")])

for idx, io_file in list(enumerate(input_files))[:]:
  input_file_path = path.join(IMAGES_PATH, io_file)
  output_file_path = path.join(EMBEDDINGS_PATH, io_file.replace(".jpg", ".json"))

  if path.isfile(output_file_path):
    continue

  if idx % 100 == 0:
    print(idx, IMAGES_PATH, io_file)

  image = PImageOps.exif_transpose(PImage.open(input_file_path).convert("RGB"))

  raw_embs = {}

  raw_embs["clip"] = Clip.get_embedding(image).tolist()
  raw_embs["efficient"] = EfficientNet.get_embedding(image).tolist()
  raw_embs["resnet"] = ResNet.get_embedding(image).tolist()
  raw_embs["vit"] = Vit.get_embedding(image).tolist()

  image_embs = {"raw": raw_embs}

  with open(output_file_path, "w", encoding="utf-8") as of:
    json.dump(image_embs, of, sort_keys=True, separators=(',',':'), ensure_ascii=False)

### Load Embeddings

In [ ]:
input_files = sorted([f for f in listdir(IMAGES_PATH) if f.endswith("jpg")])

raw_embs = {}

for idx, io_file in enumerate(input_files):
  output_file_path = path.join(EMBEDDINGS_PATH, io_file.replace(".jpg", ".json"))

  if not path.isfile(output_file_path):
    continue

  if idx % 100 == 0:
    print(idx, IMAGES_PATH, io_file)

  with open(output_file_path, "r", encoding="utf8") as f:
    image_embs = json.load(f)

  for k,e in image_embs["raw"].items():
    if k not in raw_embs:
      raw_embs[k] = []
    raw_embs[k].append(e)

### Cluster

In [ ]:
def pca_kmeans(emb_raw, n_clusters=8, n_components=128):
  mPCA = PCA(n_components=n_components)
  mCluster = KMeans(n_clusters=n_clusters)

  emb_reduced = mPCA.fit_transform(StandardScaler().fit_transform(emb_raw))
  emb_clusters = mCluster.fit_predict(emb_reduced)

  return emb_reduced, emb_clusters, mCluster.cluster_centers_

def tsne_kmeans(emb_raw, n_clusters=8, n_components=3, perplexity=30):
  mTSNE = TSNE(n_components=n_components, perplexity=perplexity)
  mCluster = KMeans(n_clusters=n_clusters)

  emb_reduced = mTSNE.fit_transform(StandardScaler().fit_transform(emb_raw))
  emb_clusters = mCluster.fit_predict(emb_reduced)

  return emb_reduced, emb_clusters, mCluster.cluster_centers_

In [ ]:
def plot_clusters(clusters, pcas, title=""):
  sizes = [0 if c < 0 else 24 for c in clusters]
  dims = pcas.shape[1]
  plot_dims = min(dims, 3)

  for i in range(plot_dims):
    for j in range(i+1, plot_dims):
      plt.scatter(pcas[:,i], pcas[:,j],
                  marker='o', s=sizes, c=clusters, alpha=0.35, cmap="tab10", edgecolors="none")
      plt.title(title)
      plt.show()

  # 3D
  if dims > 2:
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(projection='3d')
    ax.scatter(pcas[:,0], pcas[:,1], pcas[:,2],
              marker='o', s=sizes, c=clusters, alpha=0.35, cmap="tab10", edgecolors="none")
    ax.set_title(title)
    plt.show()

In [ ]:
def plot_cluster_elbows(embs, title, min_clusters=2, max_clusters=10):
  sil = []
  ncs = range(min_clusters, max_clusters+1)
  for nc in ncs:
    _pca, _clusters, _ = pca_kmeans(embs, n_clusters=nc)
    _sil = silhouette_score(_pca, _clusters)
    sil.append(_sil)

  plt.plot(ncs, sil)
  plt.title(f"{title} - Silhouette Score")
  plt.show()

In [ ]:
def plot_pca_elbows(embs, title):
  sil = []
  ncs = [min(len(embs), len(embs[0]), 2 ** i) for i in range(11)]
  ncs.append(min(len(embs), len(embs[0])))
  for nc in ncs:
    _pca, _clusters, _ = pca_kmeans(embs, n_components=nc, n_clusters=8)
    _sil = silhouette_score(_pca, _clusters)
    sil.append(_sil)

  plt.plot(ncs, sil)
  plt.title(f"{title} - Silhouette Score")
  plt.show()

In [ ]:
for m in ["clip", "efficient", "resnet", "vit"]:
  plot_cluster_elbows(raw_embs[m], m)

In [ ]:
for m in ["clip", "efficient", "resnet", "vit"]:
  plot_pca_elbows(raw_embs[m], m)

In [ ]:
for m in ["clip", "efficient", "resnet", "vit"]:
  _pca, _clusters, _ = pca_kmeans(raw_embs[m])
  plot_clusters(_clusters, _pca, title=m)

In [ ]:
clip_pca, clip_clusters, clip_centers = pca_kmeans(raw_embs["clip"])

In [ ]:
efficient_pca, efficient_clusters, efficient_centers = pca_kmeans(raw_embs["efficient"])

In [ ]:
resnet_pca, resnet_clusters, resnet_centers = pca_kmeans(raw_embs["resnet"])

In [ ]:
vit_pca, vit_clusters, vit_centers = pca_kmeans(raw_embs["vit"])

In [ ]:
input_files = sorted([f for f in listdir(IMAGES_PATH) if f.endswith("jpg")])

In [ ]:
m_pca, m_clusters, m_centers = pca_kmeans(raw_embs["resnet"])

for c in np.unique(m_clusters):
  cluster_center = m_centers[c]
  cluster_idxs = np.where(m_clusters == c)[0]
  cluster_pcas = m_pca[cluster_idxs]
  pca_center_dists = np.linalg.norm(cluster_pcas - cluster_center, axis=1)
  cluster_idxs_sorted = cluster_idxs[pca_center_dists.argsort()]

  fig, axes = plt.subplots(nrows=8, ncols=8)
  fig.set_size_inches(10, 10)
  fig.set_dpi(72)

  fig.suptitle(f"Cluster {c}")
  for ciidx, ax in enumerate(axes.flat):
    iidx = cluster_idxs_sorted[ciidx]
    img = PImage.open(path.join(IMAGES_PATH, input_files[iidx])).convert("RGB")
    img = PImageOps.exif_transpose(img).resize((128,128))
    ax.imshow(img)
    ax.axis("off")

  plt.tight_layout()
  plt.show()

### Load Clip Embeddings

In [ ]:
input_files = sorted([f for f in listdir(IMAGES_PATH) if f.endswith("jpg")])

clip_embs = {}

for idx, io_file in enumerate(input_files):
  output_file_path = path.join(EMBEDDINGS_PATH, io_file.replace(".jpg", ".json"))
  img_id = io_file.replace(".jpg", "")

  if not path.isfile(output_file_path):
    continue

  if idx % 100 == 0:
    print(idx, IMAGES_PATH, io_file)

  with open(output_file_path, "r", encoding="utf8") as f:
    image_embs = json.load(f)

  clip_embs[img_id] = {}
  clip_embs[img_id]["raw"] = [round(x, 8) for x in image_embs["raw"]["clip"]]

In [ ]:
clip_raw_vals = [e["raw"] for e in clip_embs.values()]

In [ ]:
clip_pca_vals, clip_pca_clusters, clip_pca_centers = pca_kmeans(clip_raw_vals)
plot_clusters(clip_pca_clusters, clip_pca_vals, title="clip pca")

In [ ]:
clip_tsne3_vals, clip_tsne3_clusters, clip_tsne3_centers = tsne_kmeans(clip_raw_vals, n_components=3)
plot_clusters(clip_tsne3_clusters, clip_tsne3_vals, title="clip tsne 3D")

In [ ]:
clip_tsne2_vals, clip_tsne2_clusters, clip_tsne2_centers = tsne_kmeans(clip_raw_vals, n_components=2)
plot_clusters(clip_tsne2_clusters, clip_tsne2_vals, title="clip tsne 2D")

### Grid Raster

In [ ]:
import lap
from scipy.spatial.distance import cdist

In [ ]:
side = 123

# normalize
data2d = clip_tsne2_vals - clip_tsne2_vals.min(axis=0)
data2d /= data2d.max(axis=0)

# crop
#data2d = data2d[:side*side, :]
print(data2d.shape)

In [ ]:
# grid points
xv, yv = np.meshgrid(np.linspace(0, 1, side), np.linspace(0, 1, side))
grid = np.dstack((xv, yv)).reshape(-1, 2)
print(grid.shape)

In [ ]:
_cost = cdist(grid, data2d, 'sqeuclidean')
cost = _cost * (10000000. / _cost.max())

plt.hist(cost.flatten(), bins=50)
plt.show()

In [ ]:
min_cost, row_assigns, col_assigns = lap.lapjv(np.copy(cost).astype(int), extend_cost=True)
grid_jv = grid[col_assigns]

print(col_assigns.shape)
print(grid_jv.shape)

# grid_jv has new, normalized, (x,y)

In [ ]:
plt.figure(figsize=(18, 18))
for start, end in zip(data2d, grid_jv):
  plt.arrow(start[0], start[1], end[0] - start[0], end[1] - start[1],
            head_length=0.003, head_width=0.003, color=(0,0,0,.15))
plt.show()

In [ ]:
for idx, io_file in enumerate(input_files):
  img_id = io_file.replace(".jpg", "")
  clip_embs[img_id]["cluster_distances"] = {}
  clip_embs[img_id]["cluster_distances"]["pca"] = [round(float(np.linalg.norm(clip_pca_vals[idx]-cc)), 4) for cc in clip_pca_centers]
  clip_embs[img_id]["cluster_distances"]["tsne2"] = [round(float(np.linalg.norm(clip_tsne2_vals[idx]-cc)), 4) for cc in clip_tsne2_centers]
  clip_embs[img_id]["cluster_distances"]["tsne3"] = [round(float(np.linalg.norm(clip_tsne3_vals[idx]-cc)), 4) for cc in clip_tsne3_centers]
  clip_embs[img_id]["pca"] = [round(float(x),8) for x in clip_pca_vals[idx]]
  clip_embs[img_id]["tsne2"] = [round(float(x),8) for x in clip_tsne2_vals[idx]]
  clip_embs[img_id]["tsne3"] = [round(float(x),8) for x in clip_tsne3_vals[idx]]
  clip_embs[img_id]["xy"] = [round(1.0 - float(x),8) for x in grid_jv[idx]]

In [ ]:
output_file_path = "/kaggle/working/embeddings_clip.json"
with open(output_file_path, "w", encoding="utf-8") as of:
  json.dump(clip_embs, of, sort_keys=True, separators=(',',':'), ensure_ascii=False)

In [ ]:
for img_id in clip_embs.keys():
  if "raw" in clip_embs[img_id]:
    del clip_embs[img_id]["raw"]
  if "pca" in clip_embs[img_id]:
    del clip_embs[img_id]["pca"]
  if "tsne3" in clip_embs[img_id]:
    del clip_embs[img_id]["tsne3"]

In [ ]:
# normalize tsne2d

tsne2_min = clip_tsne2_vals.min(axis=0)
tsne2_max = clip_tsne2_vals.max(axis=0)

x_min = tsne2_min[0].astype(float)
y_min = tsne2_min[1].astype(float)

x_max = tsne2_max[0].astype(float)
y_max = tsne2_max[1].astype(float)

x_range = x_max - x_min
y_range = y_max - y_min

for img_id in clip_embs.keys():
  x,y = clip_embs[img_id]["tsne2"]
  x_n = (x - x_min) / x_range
  y_n = (y - y_min) / y_range
  clip_embs[img_id]["tsne2"] = [round(x_n, 8), round(y_n, 8)]

In [ ]:
output_file_path = "/kaggle/working/embeddings_clip_2d.json"
with open(output_file_path, "w", encoding="utf-8") as of:
  json.dump(clip_embs, of, sort_keys=True, separators=(',',':'), ensure_ascii=False)

## Test Embeddings

In [ ]:
imgs = [
  PImage.open(IMAGES_PATH+"/100.jpg"),
  PImage.open(IMAGES_PATH+"/101.jpg"),
  PImage.open(IMAGES_PATH+"/1000.jpg"),
  PImage.open(IMAGES_PATH+"/1001.jpg"),
  PImage.open(IMAGES_PATH+"/1010.jpg"),
  PImage.open(IMAGES_PATH+"/1011.jpg"),
]

img = imgs[0]

for i in imgs:
  display(i.resize((128,128)))

In [ ]:
emb = ResNet.get_embedding(imgs)
print(emb.shape)

emb_diff = (emb - emb[1]).pow(2).sum(dim=1).pow(0.5)
emb_diff.argsort()

In [ ]:
emb = ResNet.get_embedding(img)
print(emb.shape)

In [ ]:
emb = EfficientNet.get_embedding(imgs)
print(emb.shape)

emb_diff = (emb - emb[1]).pow(2).sum(dim=1).pow(0.5)
emb_diff.argsort()

In [ ]:
emb = EfficientNet.get_embedding(img)
print(emb.shape)

In [ ]:
emb = Vit.get_embedding(imgs)
print(emb.shape)

emb_diff = (emb - emb[1]).pow(2).sum(dim=1).pow(0.5)
emb_diff.argsort()

In [ ]:
emb = Vit.get_embedding(img)
print(emb.shape)

In [ ]:
emb = Clip.get_embedding(imgs)
print(emb.shape)

emb_diff = (emb - emb[1]).pow(2).sum(dim=1).pow(0.5)
emb_diff.argsort()

In [ ]:
emb = Clip.get_embedding(img)
print(emb.shape)